# Setup your project folder

In [ ]:
%cd /group/30042/leoyluo/Immerse/projects/ArcNerf

# Import lib

In [ ]:
import os
import time

import numpy as np
import torch

from arcnerf.geometry.mesh import (
    extract_mesh,
    get_normals,
    get_face_centers,
    get_verts_by_faces,
    simplify_mesh
)
from arcnerf.geometry.volume import Volume
from arcnerf.models import build_model
from arcnerf.visual.plot_3d import draw_3d_components
from common.utils.cfgs_utils import load_configs
from common.utils.logger import Logger
from common.utils.model_io import load_model
from common.utils.torch_utils import torch_to_np

# Specify the model cfgs and model_pt with device

In [ ]:
cfgs_file = '/group/30042/leoyluo/Immerse/projects/ArcNerf/configs/inference.yaml'
model_pt = '/group/30042/leoyluo/Immerse/projects/ArcNerf/experiments/capture_qqtiger_nerf/checkpoints/final.pt.tar'
device = 'gpu'  # 'cpu' or 'gpu'

assert os.path.exists(cfgs_file), 'cfgs not exist at {}'.format(cfgs_file)
assert os.path.exists(model_pt), 'model file not exist at {}'.format(model_pt)

# Set up cfgs, device, models

In [ ]:
cfgs = load_configs(cfgs_file)
logger = Logger()

if torch.cuda.is_available() and device == 'gpu':
    torch.cuda.set_device(0)
    
model = build_model(cfgs, None)
model = load_model(logger, model, None, model_pt, cfgs)
if device == 'gpu':
    model.cuda()

# Set volume params and get volume

In [ ]:
n_grid = 128                 # n_grid is the num of voxel in each dim. For visual set a small num only
side = 1.5                   # if set, all len at each dim will be side
# xyz_len if you find the extract volume is not a cube
grad_dir = 'descent'         # if 'descent', sigma is larger than level in obj(NeRF), if 'ascent' is smaller(SDF)

chunk_pts_factor= 32         # process more pts together
model.set_chunk_pts(model.get_chunk_pts() * chunk_pts_factor)

# volume 
volume = Volume(n_grid, side=side)
volume_pts = volume.get_volume_pts()  # (n_grid^3, 3) pts in torch
volume_size = volume.get_voxel_size()  # (3,) tuple
volume_len = volume.get_len()  # (3,) tuple
dtype = volume_pts.dtype
if device == 'gpu':
    volume_pts = volume_pts.cuda()

# for visual
corner = torch_to_np(volume.get_corner())
bound_lines = volume.get_bound_lines()
volume_dict = {'grid_pts': corner, 'lines': bound_lines}

# Get point cloud from volume

In [ ]:
# get init sigma
time0 = time.time()
sigma, rgb = model.forward_pts_dir(volume_pts, None)
sigma, rgb = torch_to_np(sigma), torch_to_np(rgb)
print('Forward {}^3 time for model is {:.2f}s'.format(n_grid, time.time() - time0))
print('Sigma value range {:.2f}-{:.2f}'.format(sigma.min(), sigma.max()))

In [ ]:
# set sigma based on sigma output
level = 50.0                 # sigma level to extract mesh from volume  

# get valid sigma
if grad_dir == 'descent':
    valid_sigma = (sigma >= level)  # (n^3,)
else:
    valid_sigma = (sigma <= level)  # (n^3,)

# set max_pts pts, get pts and show pts
max_pts=200000

valid_pts = torch_to_np(volume_pts)[valid_sigma]  # (n_valid, 3)
valid_rgb = rgb[valid_sigma]  # (n_valid, 3)
print('Getting {} valid pts'.format(valid_pts.shape[0]))
n_pts = valid_pts.shape[0]

if n_pts > max_pts:
    print('Sample to {} pts'.format(max_pts))
    choice = np.random.choice(range(n_pts), max_pts, replace=False)
    valid_pts = valid_pts[choice]
    valid_rgb = valid_rgb[choice]

In [ ]:
# draw 3d pts
draw_3d_components(
    points=valid_pts,
    point_colors=valid_rgb,
    point_size=10,
    volume=volume_dict,
    title='valid pts({}) from volume'.format(valid_pts.shape[0]),
    plotly=True
)

# Get Mesh from volume using density

In [ ]:
# extract original mesh
sigma = sigma.reshape((n_grid, n_grid, n_grid))  # (n, n, n)
time0 = time.time()
verts, faces, _ = extract_mesh(sigma.copy(), level, volume_size, volume_len, grad_dir)
print('Extract mesh time {:.2f}s'.format(time.time() - time0))
print('Extract {} verts, {} faces'.format(verts.shape[0], faces.shape[0]))

# simplify for 3d visual, get colors
max_faces=200000
if faces.shape[0] > max_faces:
    verts, faces = simplify_mesh(verts, faces, max_faces)
    print('    Simplify mesh time {:.2f}s'.format(time.time() - time0))
    print('    Simplify {} verts, {} faces'.format(verts.shape[0], faces.shape[0]))

n_verts, n_faces = verts.shape[0], faces.shape[0]

In [ ]:
# clear gpu memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# get components like normal, color
vert_normals, face_normals = get_normals(verts, faces)
face_centers = get_face_centers(verts, faces)

# get face_colors, view point is the reverse normal
face_view_dir = -face_normals
face_center_pts = torch.tensor(face_centers, dtype=dtype)  # (n, 3)
face_view_dir = torch.tensor(face_view_dir, dtype=dtype)  # (n, 3)

# move to gpu
if device == 'gpu':
    face_center_pts = face_center_pts.cuda()
    face_view_dir = face_view_dir.cuda()

time0 = time.time()
_, face_colors = model.forward_pts_dir(face_center_pts, face_view_dir)
face_colors = torch_to_np(face_colors)
print('Get faces color for all {} faces takes {:.2f}s'.format(n_faces, time.time() - time0))

# verts from (V, 3) to (F, 3, 3)
verts_by_faces, _ = get_verts_by_faces(verts, faces, None)

In [ ]:
# draw 3d mesh
draw_3d_components(
    volume=volume_dict,
    meshes=[verts_by_faces],
    face_colors=[face_colors],
    title='Meshes ({} faces) extract from volume'.format(verts_by_faces.shape[0]),
    plotly=True
)